In [1]:
#直方图算法：把连续值离散化成k个整数，并且构造宽度为k的直方图，遍历数据后，
#将离散化后的值作为索引在直方图中累积统计量。
#Leaf-wise：每次从当前的叶子中，找到分裂增益最大的一个叶子，然后分裂，继续循环。

#lightbgm参数：
#learning_rate:学习率
#n_estimators:树的个数，迭代的次数
#max_depth:树的深度，深度大可能过拟合
#num_leaves:一棵树最大的叶子数量，num_leaves = 2^(max_depth)，但是它的值的设置应该小于 2^(max_depth)，否则可能会导致过拟合。
#min_child_samples:它的值取决于训练数据的样本个数和num_leaves,可以降低过拟合
#min_child_weight:
#reg_alpha:l1正则化
#reg_lambda：l2正则化
#colsample_bylevel, colsample_bytree, colsample_bynode ：分别表示各个层、各棵树、各个节点的列采样率
#bagging_fraction：表示每次迭代所使用的数据分数（即所占百分比，用小数表示）。将此值设置得较低，以提高训练速度。


In [62]:
import numpy as np
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer,MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import lightgbm as lgb

iris = datasets.load_iris()
X = iris.data
y = iris.target
scaler=MinMaxScaler(feature_range=(0,1))
iris = datasets.load_iris()
X=scaler.fit_transform(X)




In [63]:
def GridSearch(model,params,X,y):
    lgbr=GridSearchCV(model,params,scoring='neg_mean_squared_error',cv=5)
    lgbr.fit(X,y)
    print('GridSearchCV_best_score:',lgbr.best_score_)
    print('GridSearchCV_best_params：',lgbr.best_params_) 
    print('GridSearchCV_best_model：',lgbr.best_estimator_) 

In [64]:
params={
    'objective':'regression',
    'n_estimators':30,
    'max_depth':5,
    'min_child_samples':20,
    'reg_lambda':0.1,
    'reg_alpha':0.1,
    'metric':'rmse',
    'colsample_bytree':1,
    'subsample':0.8,
    'num_leaves':30,
    'random_state':60
}
model=lgb.LGBMRegressor(**params)
change_params={
                 'n_estimators': range(20, 200, 20),
                 'min_child_weight': range(3, 15, 2),
              #   'colsample_bytree': np.arange(0.4, 1.0),
               #  'max_depth': range(5, 15, 2),
              #   'subsample': np.arange(0.5, 1.0, 0.1),
               #  'reg_lambda': np.arange(0.1, 1.0, 0.2),
               #  'reg_alpha': np.arange(0.1, 1.0, 0.2),
                 #'min_child_samples': range(10, 30)
}
GridSearch(model,change_params,X,y)

GridSearchCV_best_score: -0.050241248298760095
GridSearchCV_best_params： {'min_child_weight': 3, 'n_estimators': 60}
GridSearchCV_best_model： LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1,
              importance_type='split', learning_rate=0.1, max_depth=5,
              metric='rmse', min_child_samples=20, min_child_weight=3,
              min_split_gain=0.0, n_estimators=60, n_jobs=-1, num_leaves=30,
              objective='regression', random_state=60, reg_alpha=0.1,
              reg_lambda=0.1, silent=True, subsample=0.8,
              subsample_for_bin=200000, subsample_freq=0)


In [65]:
#RandomSearchCV:并不是所有的参数都尝试，具有随机性
def RandomSearch(model,params,X,y):
    lgbr=RandomizedSearchCV(model,params,scoring='neg_mean_squared_error',cv=5)
    lgbr.fit(X,y)
    print('RandomizedSearchCV_best_score:',lgbr.best_score_)
    print('RandomizedSearchCV_best_params：',lgbr.best_params_) 
    print('RandomizedSearchCV_best_model：',lgbr.best_estimator_) 

In [24]:
params={
    'objective':'regression',
    'n_estimators':30,
    'max_depth':5,
    'min_child_samples':20,
    'reg_lambda':0.1,
    'reg_alpha':0.1,
    'metric':'rmse',
    'colsample_bytree':1,
    'subsample':0.8,
    'num_leaves':30,
    'random_state':60
}
model=lgb.LGBMRegressor(**params)
change_params={
                 'n_estimators': range(20, 200, 20),
                 'min_child_weight': range(3, 15, 2),
                 'colsample_bytree': np.arange(0.4, 1.0),
                 'max_depth': range(5, 15, 2),
                 'subsample': np.arange(0.5, 1.0, 0.1),
                 'reg_lambda': np.arange(0.1, 1.0, 0.2),
                 'reg_alpha': np.arange(0.1, 1.0, 0.2),
                 'min_child_samples': range(10, 30)
}
RandomSearch(model,change_params ,X,y)

RandomizedSearchCV_best_score: -0.045894939195014126
RandomizedSearchCV_best_params： {'subsample': 0.7999999999999999, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.9000000000000001, 'n_estimators': 100, 'min_child_weight': 5, 'min_child_samples': 21, 'max_depth': 9, 'colsample_bytree': 0.4}
RandomizedSearchCV_best_model： LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.4,
              importance_type='split', learning_rate=0.1, max_depth=9,
              metric='rmse', min_child_samples=21, min_child_weight=5,
              min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=30,
              objective='regression', random_state=60,
              reg_alpha=0.9000000000000001, reg_lambda=0.7000000000000001,
              silent=True, subsample=0.7999999999999999,
              subsample_for_bin=200000, subsample_freq=0)


In [55]:
#贝叶斯优化
from sklearn.model_selection import train_test_split,cross_val_score	
from bayes_opt import BayesianOptimization
def BayesianSearch(model,params):
    num_iters=30
    init_points=5
    bayes= BayesianOptimization(model, params)
    bayes.maximize(init_points=init_points,n_iter=num_iters)
    params = bayes.res
    index = []
    for i in bayes.res:
        index.append(i['target'])
    max_index = index.index(max(index))

    print(params[max_index])
    return max_index
        

In [56]:

def evaluate(min_child_samples, min_child_weight, colsample_bytree, max_depth, subsample, reg_alpha, reg_lambda):
    param={
        'objective': 'regression',
        'n_estimators': 50,
        'metric': 'rmse',
        'random_state': 60
    }#固定参数
    param['min_child_samples'] = int(min_child_samples)
    param['min_child_weight'] = int(min_child_weight)
    param['colsample_bytree'] = float(colsample_bytree),
    param['max_depth'] = int(max_depth),
    param['subsample'] = float(subsample),
    param['reg_lambda'] = float(reg_lambda),
    param['reg_alpha'] = float(reg_alpha),
    #超参数
    cvs=cross_val_score(lgb.LGBMRegressor(**param),X,y,scoring='neg_mean_squared_error',cv=5).mean()
    return cvs


    

In [57]:
change_params={
               
                 'min_child_weight': (3, 15),
                 'colsample_bytree': (0.4, 1.0),
                 'max_depth': (5, 15),
                 'subsample': (0.5, 1.0),
                 'reg_lambda': (0.1, 1.0),
                 'reg_alpha': (0.1, 1.0),
                 'min_child_samples':(10, 30)
}

In [58]:
BayesianSearch(evaluate,change_params)

|   iter    |  target   | colsam... | max_depth | min_ch... | min_ch... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        | -0.07297  |  0.4831   |  7.804    |  27.27    |  14.06    |  0.9263   |  0.1577   |  0.7374   |
|  2        | -0.06757  |  0.9476   |  11.05    |  27.08    |  12.49    |  0.3119   |  0.5761   |  0.7068   |
|  3        | -0.07113  |  0.5115   |  14.48    |  28.21    |  12.15    |  0.4027   |  0.4682   |  0.5027   |
|  4        | -0.04918  |  0.7638   |  7.86     |  18.5     |  14.62    |  0.1796   |  0.4666   |  0.7052   |
|  5        | -0.05014  |  0.4811   |  11.93    |  15.43    |  14.86    |  0.2225   |  0.5308   |  0.7365   |
|  6        | -0.0548   |  0.4125   |  5.743    |  10.41    |  3.518    |  0.7695   |  0.7817   |  0.8266   |
|  7        | -0.05262  |  0.4597   |  5.417    |  10.26    |  14.8     |  0.4896   |  0.9789   |  0.5665   |
|  8      

20